In [7]:
import pandas as pd
import numpy as np
import xlsxwriter

import openpyxl
import mysql.connector

In [11]:
# Connecting to the server
def open_conn():
    conn = mysql.connector.connect(user='root', password='1985',
                              host='127.0.0.1', database='ulb_data',
                              auth_plugin='mysql_native_password')
    print("DB Connection Opened!")
    return conn
 
# Disconnecting from the server
def close_con(conn):
    conn.close()
    print("DB connection closed")

In [9]:
query = "select ulb_hrms_id, ulb_name from ulbs where ulb_name like '%robert%'"

In [12]:
conn = open_conn()
mycursor = conn.cursor()
mycursor.execute(query)
myresult = mycursor.fetchall()
for x in myresult:
  print(x)

close_con(conn)

DB Connection Opened!
('0711CU0001', 'Robertsonpet(KGF)')
DB connection closed


In [17]:
path = "Budget_Files_2024-25/Appendix-B_2024-25_For_CCs.xlsx"

In [58]:
df = pd.DataFrame(columns=['ulb_name', 'ulb_hrms_code', 'emp_id', 'pension_type', 'pran', 'emp_name', 'gender', 'designation',
                           'payscale_min', 'payscale_max', 'basic_pay', 'basic_pay_for_year', 'increment_month', 'increment_rate',
                           'increment_for_year', 'basic_incrnt_total', 'da_for_year', 'hra_for_year', 'ma_for_year',
                           'ua_pk_spcl_for_year','cca_for_year', 'other_allowance_type', 'oa_amount_for_year',
                           'surrender_leave_encash','nps_contr_for_year', 'total'])

,ulb_name,ulb_hrms_code,emp_id,pension_type,pran,emp_name,gender,designation,payscale_min,payscale_max,...,da_for_year,hra_for_year,ma_for_year,ua_pk_spcl_for_year,cca_for_year,other_allowance_type,oa_amount_for_year,surrender_leave_encash,nps_contr_for_year,total


In [18]:
# load excel with its path
wrkbk = openpyxl.load_workbook(path, data_only=True)
  
sh = wrkbk.active

# sh.cell(row=4,column=4).value
# sh.cell(row=4,column=24).value

# ulb_name = sh['D4'].value
# ulb_hrms_code = sh['X4'].value

# iterate through excel and display data
list = []
for i in range(9, 3008):
    list.insert(0,sh.cell(4,4).value)
    list.insert(1,sh.cell(4,24).value)
    if sh.cell(row=i, column=2).value == None:
        break
    else:
        for j in range(2, 26):
            if j==26:
                cell_obj = sh.cell(row=i, column=j).value
                break
            else:
                cell_obj = sh.cell(row=i, column=j).value
                if type(cell_obj) == 'int':
                    list.insert(j, cell_obj)
                elif type(cell_obj) == 'str':
                    list.insert(j,"'" + cell_obj + "'")
                else:
                    list.insert(j, cell_obj)
    
    print(list)
    df.loc[len(df)] = list
    list=[]

911254678,OPS,None,sdfsdfsdfd,F,PK,17000,28950,17000,204000,2024-07-01 00:00:00,400,3600,207600,91344,16608,2400,8400,4800,PP,1200,13148,-,345500
911257426,NPS,110145623852,dsdfsdfsd,M,CAUO,74400,109600,74400,892800,2025-01-01 00:00:00,1900,5700,898500,395340,71880,0,0,6000,SFN,1500,56905,181137.6,1611262.6
911121651,NPS,123456789123,sxdfcsdfd,F,RO,43100,83900,43100,517200,2025-01-01 00:00:00,1100,3300,520500,229020,41640,0,0,3434,SFN,1500,32965,104932.8,933991.8


In [55]:
list = []
for i in range(9, 3008):
    list.insert(0,sh.cell(4,4).value)
    list.insert(1,sh.cell(4,24).value)
    if sh.cell(row=i, column=2).value == None:
        break
    else:
        for j in range(2, 26):
            if j==26:
                cell_obj = sh.cell(row=i, column=j).value
                break
            else:
                cell_obj = sh.cell(row=i, column=j).value
                if type(cell_obj) == 'int':
                    list.insert(j, cell_obj)
                elif type(cell_obj) == 'str':
                    list.insert(j,"'" + cell_obj + "'")
                else:
                    list.insert(j, cell_obj)
    
    print(list)
    df.loc[len(df)] = list
    list=[]

['Tumakuru', '2201CC0123', 911254678, 'OPS', None, 'sdfsdfsdfd', 'F', 'PK', 17000, 28950, 17000, 204000, datetime.datetime(2024, 7, 1, 0, 0), 400, 3600, 207600, 91344, 16608, 2400, 8400, 4800, 'PP', 1200, 13148, '-', 345500]
['Tumakuru', '2201CC0123', 911257426, 'NPS', 110145623852, 'dsdfsdfsd', 'M', 'CAUO', 74400, 109600, 74400, 892800, datetime.datetime(2025, 1, 1, 0, 0), 1900, 5700, 898500, 395340, 71880, 0, 0, 6000, 'SFN', 1500, 56905, 181137.6, 1611262.6]
['Tumakuru', '2201CC0123', 911121651, 'NPS', 123456789123, 'sxdfcsdfd', 'F', 'RO', 43100, 83900, 43100, 517200, datetime.datetime(2025, 1, 1, 0, 0), 1100, 3300, 520500, 229020, 41640, 0, 0, 3434, 'SFN', 1500, 32965, 104932.8, 933991.8]


In [56]:
print(df)

   ulb_name ulb_hrms_code     emp_id pension_type          pran    emp_name  \
0  Tumakuru    2201CC0123  911254678          OPS          None  sdfsdfsdfd   
1  Tumakuru    2201CC0123  911257426          NPS  110145623852   dsdfsdfsd   
2  Tumakuru    2201CC0123  911121651          NPS  123456789123   sxdfcsdfd   
3  Tumakuru    2201CC0123  911254678          OPS          None  sdfsdfsdfd   
4  Tumakuru    2201CC0123  911257426          NPS  110145623852   dsdfsdfsd   
5  Tumakuru    2201CC0123  911121651          NPS  123456789123   sxdfcsdfd   

  gender designation  payscale_min  payscale_max  ...  da_for_year  \
0      F          PK         17000         28950  ...        91344   
1      M        CAUO         74400        109600  ...       395340   
2      F          RO         43100         83900  ...       229020   
3      F          PK         17000         28950  ...        91344   
4      M        CAUO         74400        109600  ...       395340   
5      F          RO      